In [1]:
import pandas as pd
import itertools
from collections import defaultdict, Counter

# 1. Carregamento dos dados
# Certifique-se de que os caminhos dos arquivos estão corretos
authors_df = pd.read_csv('database/authorships.csv')
works_df = pd.read_csv('database/works.csv')

# 2. Pré-processamento e Junção
# Precisamos da data de publicação associada a cada autoria
# Fazemos um merge entre a tabela de autores e a tabela de trabalhos
merged_df = authors_df.merge(
    works_df[['id', 'publication_date']], 
    left_on='work_id', 
    right_on='id'
)

# Converter coluna de data para datetime para ordenação correta
merged_df['publication_date'] = pd.to_datetime(merged_df['publication_date'], errors='coerce')

# Remover linhas sem data (se houver) para garantir a ordem temporal
merged_df = merged_df.dropna(subset=['publication_date'])

# 3. Corte Temporal (80% Treino / 20% Teste)
# É importante dividir baseado nas OBRAS (works), não nas linhas, para não quebrar uma obra no meio
unique_works = merged_df[['work_id', 'publication_date']].drop_duplicates().sort_values('publication_date')

# Índice de corte (80%)
split_idx = int(len(unique_works) * 0.8)

# Pegamos os IDs das obras que compõem os 80% mais antigos
train_work_ids = set(unique_works.iloc[:split_idx]['work_id'])

# Filtramos o DataFrame principal para manter apenas essas obras no treino
train_df = merged_df[merged_df['work_id'].isin(train_work_ids)].copy()

print(f"Total de trabalhos: {len(unique_works)}")
print(f"Trabalhos no Treino (80%): {len(train_work_ids)}")

# 4. Construção do Grafo de Coautoria (Apenas dados de Treino)
# Usamos um dicionário onde a chave é o ID do autor e o valor é um conjunto (set) de coautores
coauthors_graph = defaultdict(set)

# Agrupamos por trabalho para identificar quem escreveu junto
for work_id, group in train_df.groupby('work_id'):
    authors_in_paper = group['author_id'].tolist()
    
    # Só há coautoria se houver mais de 1 autor
    # (Otimização: papers com muitos autores (ex: 500+) podem deixar o loop lento, 
    # você pode colocar um limite 'if len(authors_in_paper) < 50:' se necessário)
    if len(authors_in_paper) > 1:
        # Gera todas as permutações de pares (A, B) e (B, A)
        for u, v in itertools.permutations(authors_in_paper, 2):
            coauthors_graph[u].add(v)

# 5. Função de Recomendação (Common Neighbors)
def recommend_coauthors(author_id, graph, top_n=5):
    """
    Recomenda autores que não são vizinhos diretos, mas compartilham vizinhos.
    Ordena pelo número de vizinhos compartilhados.
    """
    if author_id not in graph:
        return [] # Autor não tem coautorias no conjunto de treino
    
    current_coauthors = graph[author_id]
    candidates = []
    
    # Para cada coautor que o autor A já tem...
    for neighbor in current_coauthors:
        # Olhamos os coautores desse coautor (vizinhos dos vizinhos)
        neighbors_of_neighbor = graph.get(neighbor, set())
        
        for candidate in neighbors_of_neighbor:
            # Critérios de exclusão:
            # 1. Não pode ser o próprio autor
            # 2. Não pode ser alguém com quem ele JÁ escreveu (vizinho direto)
            if candidate != author_id and candidate not in current_coauthors:
                candidates.append(candidate)
    
    # Contamos a frequência (quantos 'neighbor' em comum levaram a este 'candidate')
    candidate_counts = Counter(candidates)
    
    # Retorna os top N ordenados por maior número de coautores em comum
    return candidate_counts.most_common(top_n)

# --- Exemplo de Uso ---

# Escolha um autor para testar (substitua pelo ID desejado)
# Exemplo: pegando o primeiro autor que aparece no dataset de treino
sample_author_id = train_df['author_id'].iloc[0] 
sample_author_name = train_df[train_df['author_id'] == sample_author_id]['author_name'].iloc[0]

recommendations = recommend_coauthors(sample_author_id, coauthors_graph)

print(f"\nRecomendações para {sample_author_name} ({sample_author_id}):")
for rec_id, score in recommendations:
    # Busca o nome do recomendado para exibir
    rec_name = authors_df[authors_df['author_id'] == rec_id]['author_name'].iloc[0]
    print(f"Autor: {rec_name} | Coautores em comum: {score}")

Total de trabalhos: 3574
Trabalhos no Treino (80%): 2859

Recomendações para Luciana Principal Antunes (https://openalex.org/A5069790395):
Autor: Jesus Aparecido Ferro | Coautores em comum: 12
Autor: Dirce Maria Carraro | Coautores em comum: 10
Autor: Leandro Márcio Moreira | Coautores em comum: 10
Autor: Suzan Pantaroto de Vasconcellos | Coautores em comum: 10
Autor: Aline Da Silva | Coautores em comum: 9


In [2]:
import pandas as pd
import itertools
import numpy as np
from collections import defaultdict, Counter

# --- ETAPAS ANTERIORES (CARREGAMENTO E SPLIT) ---
# (Repetindo o setup essencial para o código ser autossuficiente)

authors_df = pd.read_csv('database/authorships.csv')
works_df = pd.read_csv('database/works.csv')

merged_df = authors_df.merge(
    works_df[['id', 'publication_date']], 
    left_on='work_id', right_on='id'
)
merged_df['publication_date'] = pd.to_datetime(merged_df['publication_date'], errors='coerce')
merged_df = merged_df.dropna(subset=['publication_date'])

unique_works = merged_df[['work_id', 'publication_date']].drop_duplicates().sort_values('publication_date')
split_idx = int(len(unique_works) * 0.8)

# IDs de Treino e Teste
train_work_ids = set(unique_works.iloc[:split_idx]['work_id'])
test_work_ids = set(unique_works.iloc[split_idx:]['work_id'])

train_df = merged_df[merged_df['work_id'].isin(train_work_ids)]
test_df = merged_df[merged_df['work_id'].isin(test_work_ids)]

# --- 1. CONSTRUÇÃO DOS GRAFOS ---

# Função auxiliar para criar grafo
def build_graph(df):
    graph = defaultdict(set)
    for _, group in df.groupby('work_id'):
        authors = group['author_id'].tolist()
        if len(authors) > 1:
            for u, v in itertools.permutations(authors, 2):
                graph[u].add(v)
    return graph

# Grafo de Treino (O que o modelo conhece)
train_graph = build_graph(train_df)

# Grafo de Teste (O futuro real)
test_graph_raw = build_graph(test_df)

# --- 2. DEFINIR O GROUND TRUTH (NOVAS RELAÇÕES APENAS) ---
# Queremos prever conexões que NÃO existiam no treino e apareceram no teste.
test_ground_truth = defaultdict(set)

for author, coauthors in test_graph_raw.items():
    # Pega quem o autor colaborou no futuro
    future_coauthors = coauthors
    # Remove quem ele JÁ conhecia no passado (não é predição nova)
    past_coauthors = train_graph.get(author, set())
    
    new_links = future_coauthors - past_coauthors
    
    if new_links:
        test_ground_truth[author] = new_links

print(f"Autores no treino: {len(train_graph)}")
print(f"Autores com novas conexões no teste (para avaliar): {len(test_ground_truth)}")

# --- 3. FUNÇÃO DE RECOMENDAÇÃO (A mesma anterior) ---
def recommend_coauthors(author_id, graph, top_n=10):
    if author_id not in graph: return []
    current_coauthors = graph[author_id]
    candidates = []
    for neighbor in current_coauthors:
        neighbors_of_neighbor = graph.get(neighbor, set())
        for candidate in neighbors_of_neighbor:
            if candidate != author_id and candidate not in current_coauthors:
                candidates.append(candidate)
    return [c[0] for c in Counter(candidates).most_common(top_n)]

# --- 4. AVALIAÇÃO (PRECISION, RECALL, F1) ---

K = 10  # Avaliando o Top-10 recomendações
precisions = []
recalls = []

# Avaliamos APENAS autores que realmente formaram novas conexões
# (Não faz sentido avaliar quem parou de publicar ou só trabalhou com velhos amigos)
for author_id, actual_new_coauthors in test_ground_truth.items():
    
    # O modelo faz a predição baseada APENAS no grafo de treino
    recommendations = recommend_coauthors(author_id, train_graph, top_n=K)
    
    # Se o modelo não recomendou ninguém (ex: autor isolado), precisão é 0
    if not recommendations:
        precisions.append(0)
        recalls.append(0)
        continue

    # Quantos acertos (Interseção entre Recomendados e Reais)
    hits = len(set(recommendations) & actual_new_coauthors)
    
    # Precision: Dos que eu recomendei, quantos eram verdadeiros?
    p = hits / len(recommendations)
    
    # Recall: Dos que existiam de verdade, quantos eu encontrei?
    r = hits / len(actual_new_coauthors)
    
    precisions.append(p)
    recalls.append(r)

# --- 5. RESULTADOS MÉDIOS ---
avg_precision = np.mean(precisions)
avg_recall = np.mean(recalls)

# F1 Score (Média harmônica)
if (avg_precision + avg_recall) > 0:
    f1_score = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall)
else:
    f1_score = 0

print("-" * 30)
print(f"Resultados da Avaliação (Top-{K}):")
print(f"Precision: {avg_precision:.4f} (De cada 10 sugeridos, ~{avg_precision*10:.1f} são aceitos)")
print(f"Recall:    {avg_recall:.4f} (O modelo encontra ~{avg_recall*100:.1f}% dos novos parceiros)")
print(f"F1-Score:  {f1_score:.4f}")
print("-" * 30)

Autores no treino: 7806
Autores com novas conexões no teste (para avaliar): 2914
------------------------------
Resultados da Avaliação (Top-10):
Precision: 0.0075 (De cada 10 sugeridos, ~0.1 são aceitos)
Recall:    0.0095 (O modelo encontra ~0.9% dos novos parceiros)
F1-Score:  0.0084
------------------------------
